In [1]:
import time
source_text = "αυτή είναι μια πάρα πολύ δύσκολη μέρα"


In [2]:
# from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
# tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# encoded_zh = tokenizer(source_text, return_tensors="pt")
# generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("en"))
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [3]:
# t1 = (time.time() * 1000)
# generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("en"))
# dt = (time.time() * 1000)-t1
# # print(pred)
# print(dt,"ms")
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

In [4]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, pipeline, AutoProcessor

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
translator = pipeline("translation_el_to_en",model=model, tokenizer=tokenizer, device=0)
translator(source_text)[0]['translation_text']

/home/giorgospap/onnx/onnx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'This is a very difficult day.'

In [5]:
t1 = (time.time() * 1000)
pred = translator(source_text)[0]['translation_text']
dt = (time.time() * 1000)-t1
print(pred)
print(dt,"ms")

This is a very difficult day.
89.31591796875 ms


In [6]:
import onnxruntime as ort
ort.get_device()
ort.get_available_providers()

['TensorrtExecutionProvider',
 'CUDAExecutionProvider',
 'AzureExecutionProvider',
 'CPUExecutionProvider']

In [7]:
from transformers import AutoConfig, AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForSeq2SeqLM
# from optimum.pipelines import pipeline
model_onnx_path = "/home/giorgospap/onnx/onnx2"
model_base_id = "facebook/m2m100_418M"
model_onnx = ORTModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M",export=True)
tokenizers = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
# model_onnx.save_pretrained(model_onnx_path)

Framework not specified. Using pt to export to ONNX.


KeyboardInterrupt: 

In [ ]:

translator = pipeline("translation_el_to_en",model=model_onnx,tokenizer=tokenizers, device=0)


2023-11-22 13:04:53.552924714 [W:onnxruntime:, session_state.cc:1162 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-11-22 13:04:53.552939127 [W:onnxruntime:, session_state.cc:1164 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-11-22 13:04:55.815517639 [W:onnxruntime:, session_state.cc:1162 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-11-22 13:04:55.815543306 [W:onnxruntime:, session_state.cc:1164 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-11-22 13:04:57.405531936 [W:onnxrun

In [ ]:
t1 = (time.time() * 1000)
pred = translator(source_text)[0]['translation_text']
dt = (time.time() * 1000)-t1
print(pred)
print(dt,"ms")

This is a very difficult day.
152.038330078125 ms


In [8]:
from optimum.onnxruntime import (
    AutoOptimizationConfig, ORTOptimizer, ORTModelForSeq2SeqLM
)
save_dir = "distilbert_optimized"
model_base_id = "facebook/m2m100_418M"

# Load a PyTorch model and export it to the ONNX format
model = ORTModelForSeq2SeqLM.from_pretrained(model_base_id, export=True)

# Create the optimizer
optimizer = ORTOptimizer.from_pretrained(model)

# Define the optimization strategy by creating the appropriate configuration
optimization_config = AutoOptimizationConfig.O2()

# Optimize the model
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

Framework not specified. Using pt to export to ONNX.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.1.1+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
/home/giorgospap/onnx/onnx/lib/python3.10/site-packages/transformers/models/m2m_100/modeling_m2m_100.py:141: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if max_pos > self.weights.size(0):
/home/giorgospap/onnx/onnx/lib/python3.10/site-packages/transformers/models/m2m_100/modeling_m2m_100.py:266: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might n

: 

In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import OptimizationConfig, ORTOptimizer, ORTModelForSeq2SeqLM

model_id = "sshleifer/distilbart-cnn-12-6"
save_dir = "distilbart_optimized"

# Load a PyTorch model and export it to the ONNX format
model = ORTModelForSeq2SeqLM.from_pretrained(model_id, export=True)

# Create the optimizer
optimizer = ORTOptimizer.from_pretrained(model)

# Define the optimization strategy by creating the appropriate configuration
optimization_config = OptimizationConfig(
    optimization_level=2,
    enable_transformers_specific_optimizations=True,
    optimize_for_gpu=False,
)

# Optimize the model
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
optimized_model = ORTModelForSeq2SeqLM.from_pretrained(save_dir)
tokens = tokenizer("This is a sample input", return_tensors="pt")
outputs = optimized_model.generate(**tokens)

In [1]:
from onnxruntime.transformers.optimizer import optimize_model
from pathlib import Path
# Create a PATH to save the model
model_onnx_path = "/home/giorgospap/onnx/onnx2"
model_onnx_path = Path(model_onnx_path)

# Create a PATH to save the optimized model
optimized_onnx_path = model_onnx_path / "decoder_with_past_optimized.onnx"

# Optimize the model using ORT transformer optimizer with its default values
optimized_model = optimize_model(
    input=str(model_onnx_path / "decoder_with_past_model.onnx"),
    use_gpu=True
)

# Save the model
optimized_model.save_model_to_file(optimized_onnx_path)

2023-11-20 14:53:00.100379069 [W:onnxruntime:, session_state.cc:1162 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-11-20 14:53:00.100399280 [W:onnxruntime:, session_state.cc:1164 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [9]:
# from optimum.exporters.onnx import main_export
# from optimum.exporters.onnx.model_configs import WhisperOnnxConfig
# from transformers import AutoConfig

# from optimum.exporters.onnx.base import ConfigBehavior
# from typing import Dict

# class CustomWhisperOnnxConfig(WhisperOnnxConfig):

#     def outputs(self) -> Dict[str, Dict[int, str]]:
#         common_outputs = super().outputs

#         if self._behavior is ConfigBehavior.ENCODER:
#             for i in range(self._config.encoder_layers):
#                 common_outputs[f"encoder_attentions.{i}"] = {0: "batch_size"}
#         elif self._behavior is ConfigBehavior.DECODER:
#             for i in range(self._config.decoder_layers):
#                 common_outputs[f"decoder_attentions.{i}"] = {
#                     0: "batch_size",
#                     2: "decoder_sequence_length",
#                     3: "past_decoder_sequence_length + 1"
#                 }
#             for i in range(self._config.decoder_layers):
#                 common_outputs[f"cross_attentions.{i}"] = {
#                     0: "batch_size",
#                     2: "decoder_sequence_length",
#                     3: "encoder_sequence_length_out"
#                 }

#         return common_outputs


#     def torch_to_onnx_output_map(self):
#         if self._behavior is ConfigBehavior.ENCODER:
#             # The encoder export uses WhisperEncoder that returns the key "attentions"
#             return {"attentions": "encoder_attentions"}
#         else:
#             return {}

# model_id = "openai/whisper-tiny.en"
# config = AutoConfig.from_pretrained(model_id)

# custom_whisper_onnx_config = CustomWhisperOnnxConfig(
#         config=config,
#         task="automatic-speech-recognition",
# )

# encoder_config = custom_whisper_onnx_config.with_behavior("encoder")
# decoder_config = custom_whisper_onnx_config.with_behavior("decoder", use_past=False)
# decoder_with_past_config = custom_whisper_onnx_config.with_behavior("decoder", use_past=True)

# custom_onnx_configs={
#     "encoder_model": encoder_config,
#     "decoder_model": decoder_config,
#     "decoder_with_past_model": decoder_with_past_config,
# }

# main_export(
#     model_id,
#     output="custom_whisper_onnx",
#     no_post_process=True,
#     model_kwargs={"output_attentions": True},
#     custom_onnx_configs=custom_onnx_configs
# )

In [62]:
from optimum.exporters.onnx import main_export
from optimum.exporters.onnx.model_configs import TextSeq2SeqOnnxConfig
from transformers import AutoConfig
from optimum.utils import NormalizedSeq2SeqConfig

from optimum.exporters.onnx.base import ConfigBehavior
from typing import Dict

class CustomOnnxConfig(TextSeq2SeqOnnxConfig):
    NORMALIZED_CONFIG_CLASS = NormalizedSeq2SeqConfig
    @property
    def outputs(self) -> Dict[str, Dict[int, str]]:
        common_outputs = super().outputs

        if self._behavior is ConfigBehavior.ENCODER:
            for i in range(self._config.encoder_layers):
                common_outputs[f"encoder_attentions.{i}"] = {0: "batch_size"}
        elif self._behavior is ConfigBehavior.DECODER:
            for i in range(self._config.decoder_layers):
                common_outputs[f"decoder_attentions.{i}"] = {
                    0: "batch_size",
                    2: "decoder_sequence_length",
                    3: "past_decoder_sequence_length + 1"
                }
            for i in range(self._config.decoder_layers):
                common_outputs[f"cross_attentions.{i}"] = {
                    0: "batch_size",
                    2: "decoder_sequence_length",
                    3: "encoder_sequence_length_out"
                }

        return common_outputs

    @property
    def torch_to_onnx_output_map(self):
        if self._behavior is ConfigBehavior.ENCODER:
            # The encoder export uses WhisperEncoder that returns the key "attentions"
            return {"attentions": "encoder_attentions"}
        else:
            return {}

model_id = "facebook/m2m100_418M"
config = AutoConfig.from_pretrained(model_id)

In [63]:
custom_onnx_config = CustomOnnxConfig(
        config=config,
        task="text2text-generation"
)

In [64]:
encoder_config = custom_onnx_config.with_behavior("encoder")
decoder_config = custom_onnx_config.with_behavior("decoder", use_past=False)
decoder_with_past_config = custom_onnx_config.with_behavior("decoder", use_past=True)

custom_onnx_configs={
    "encoder_model": encoder_config,
    "decoder_model": decoder_config,
    "decoder_with_past_model": decoder_with_past_config,
}

In [66]:
main_export(
    model_id,
    output="custom_whisper_onnx",
    no_post_process=True,
    model_kwargs={"output_attentions": True},
    custom_onnx_configs=custom_onnx_configs
)

Framework not specified. Using pt to export to ONNX.
Automatic task detection to text2text-generation-with-past (possible synonyms are: seq2seq-lm-with-past).
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 2.1.1+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
/home/giorgospap/onnx/onnx/lib/python3.10/site-packages/transformers/models/m2m_100/modeling_m2m_100.py:141: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if max_pos > self.weights.size(0):
/home/giorgospap/onnx/onnx/lib/python3.10/site-packages/transformers/models/m2m_100/modeling_m2m_100.py:266: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of P

In [72]:
# from optimum.onnxruntime import ORTModelForCustomTasks
# model = ORTModelForCustomTasks.from_pretrained(model_id="custom_whisper_onnx", config=custom_onnx_configs, model_save_dir= "/home/giorgospap/onnx/test/")


In [ ]:
# from transformers import AutoTokenizer
# from optimum.onnxruntime import ORTModelForCustomTasks

# tokenizer = AutoTokenizer.from_pretrained("optimum/sbert-all-MiniLM-L6-with-pooler")
# model = ORTModelForCustomTasks.from_pretrained("optimum/sbert-all-MiniLM-L6-with-pooler")

# inputs = tokenizer("I love burritos!", return_tensors="np")

# outputs = model(**inputs)
# last_hidden_state = outputs.last_hidden_state
# pooler_output = outputs.pooler_output

In [1]:
import onnx
from onnxruntime import InferenceSession
onnx_enc_path = "encoder_model.onnx"
onnx_dec_path = "decoder_model.onnx"
onnx.checker.check_model(onnx_enc_path)
onnx.checker.check_model(onnx_dec_path)
# Start onnxruntime sessions
enc_session = InferenceSession(onnx_enc_path)
dec_session = InferenceSession(onnx_dec_path)

In [2]:
from onnxt5 import GenerativeT5
from onnxt5.api import get_encoder_decoder_tokenizer
dec_session, enc_session, tokenizer = get_encoder_decoder_tokenizer()
generative_t5 = GenerativeT5(enc_session, dec_session, tokenizer, onnx=True)

/home/giorgospap/onnx/onnx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyError: 'content-length'

In [10]:
encoder_hidden_states = enc_session.run(None, input_feed={"input_features":source_text})[0]

ValueError: Required inputs (['input_ids', 'attention_mask']) are missing from input feed (['input_features']).

In [ ]:
import numpy as np
input_ids = np.array(processor.get_decoder_prompt_ids(), dtype=np.int64)

input_feed = {"input_ids":input_ids, "encoder_hidden_states":encoder_hidden_states}

out = dec_session.run(None, input_feed=input_feed)